### 座標情報をpostで受け取る
### 接触等を見つける
### requestが来たら全部のエージェントの位置情報を返す
### シェーダー処理とかはローカルで行ってもらう。（見える一とかに寄る？）

In [ ]:
import socket
import time
import numpy as np
import json
db = {}

PORT = 4424
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('127.0.0.1', PORT))
    s.listen(1)
    while True:
        conn, addr = s.accept()
        with conn:
            while True:
                try:
                    rec = conn.recv(65536)
                except:
                    continue
                if not rec:
                    break
                rec = json.loads(rec)
                if rec['queue'] == 'post':
                    data = rec['data']
                    if not data['name'] in db.keys():
                        res = json.dumps({
                            'message': 'hello '+data['name']+' san.'}).encode()
                    conn.sendall(res)
                    db[data['name']] = data['vertices']
                elif rec['queue'] == 'get':
                    res = json.dumps({'data': db}).encode()
                    conn.sendall(res)

- ワールドのレンダリングは行う。
- 全部を一つのfigureにまとめてプロットしておく。
- camera座標を受け取ると、responseを返す・